In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

/Users/xiaojiezhou/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

# Changes
* v5 : try to optimise the LGBM params (go below in lgbm params section to see changes)
* v4 : add df, X_train deletion before training step --> increasing train sample without memeroy issues

<font color="red" size="5">If you appreciate the effort We're putting in, please upvote us :) </font>

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [3]:
pd.options.display.max_columns = 50

In [4]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [5]:
import os
os.chdir('/Users/xiaojiezhou/Documents/DataScience/Kaggle/M5')
os.getcwd()

'/Users/xiaojiezhou/Documents/DataScience/Kaggle/M5'

In [6]:
ls rawdata


M5-Competitors-Guide-Final-10-March-2020.docx
calendar.csv
calendar.csv.zip
calendar.feather
sales_train_validation.csv*
sales_train_validation.csv.zip
sales_train_validation.feather
sample_submission.csv*
sample_submission.csv.zip
sample_submission.feather
sell_prices.csv*
sell_prices.csv.zip
sell_prices.feather


In [7]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("rawdata/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv("rawdata/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv("rawdata/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [8]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [9]:
FIRST_DAY = 350 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [10]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

CPU times: user 29.8 s, sys: 9.21 s, total: 39 s
Wall time: 33.1 s


(40718219, 22)

In [11]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,3.97
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,4.34
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,2.48
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,0.50
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,1.77


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory us

In [13]:
%%time

create_fea(df)
df.shape

CPU times: user 2min 26s, sys: 23.3 s, total: 2min 49s
Wall time: 2min 15s


(40718219, 31)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
 22  lag_7         float32       
 23  lag_28        float32       
 

In [15]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,3.97,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,4.34,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,2.48,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,0.50,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,1.77,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13


In [16]:
df.dropna(inplace = True)
df.shape

(39041269, 31)

In [17]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [18]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [19]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 20 s, sys: 8.01 s, total: 28 s
Wall time: 24.1 s


In [20]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

20

In [21]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [22]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/Users/xiaojiezhou/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/xiaojiezhou/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	valid_0's rmse: 2.92019
[40]	valid_0's rmse: 2.60363
[60]	valid_0's rmse: 2.52382
[80]	valid_0's rmse: 2.50063
[100]	valid_0's rmse: 2.49131
[120]	valid_0's rmse: 2.48245
[140]	valid_0's rmse: 2.47469
[160]	valid_0's rmse: 2.46946
[180]	valid_0's rmse: 2.46108
[200]	valid_0's rmse: 2.4524
[220]	valid_0's rmse: 2.44751
[240]	valid_0's rmse: 2.44195
[260]	valid_0's rmse: 2.43743
[280]	valid_0's rmse: 2.43152
[300]	valid_0's rmse: 2.42651
[320]	valid_0's rmse: 2.42239
[340]	valid_0's rmse: 2.41761
[360]	valid_0's rmse: 2.4144
[380]	valid_0's rmse: 2.40824
[400]	valid_0's rmse: 2.40404
[420]	valid_0's rmse: 2.40048
[440]	valid_0's rmse: 2.39699
[460]	valid_0's rmse: 2.39416
[480]	valid_0's rmse: 2.38984
[500]	valid_0's rmse: 2.38747
[520]	valid_0's rmse: 2.38443
[540]	valid_0's rmse: 2.38124
[560]	valid_0's rmse: 2.37941
[580]	valid_0's rmse: 2.37691
[600]	valid_0's rmse: 2.37381
[620]	valid_0's rmse: 2.3721
[640]	valid_0's rmse: 2.37034
[660]	valid_0's rmse: 2.36851
[680]	valid_0's r

Flat Mean-14_sub.csv      Flat Median-7_sub.csv     Rolling Median-14_sub.csv
Flat Mean-28_sub.csv      PyProg                    Rolling Median-28_sub.csv
Flat Mean-56_sub.csv      RProg                     Rolling Median-7_sub.csv
Flat Mean-7_sub.csv       Rolling Mean-14_sub.csv   adata
Flat Median-14_sub.csv    Rolling Mean-28_sub.csv   catboost_info
Flat Median-28_sub.csv    Rolling Mean-56_sub.csv   rawdata
Flat Median-56_sub.csv    Rolling Mean-7_sub.csv    results


In [26]:
m_lgb.save_model("model.lgb")
!ls

Flat Mean-14_sub.csv      PyProg                    Rolling Median-56_sub.csv
Flat Mean-28_sub.csv      RProg                     Rolling Median-7_sub.csv
Flat Mean-56_sub.csv      Rolling Mean-14_sub.csv   adata
Flat Mean-7_sub.csv       Rolling Mean-28_sub.csv   catboost_info
Flat Median-14_sub.csv    Rolling Mean-56_sub.csv   model.lgb
Flat Median-28_sub.csv    Rolling Mean-7_sub.csv    rawdata
Flat Median-56_sub.csv    Rolling Median-14_sub.csv results
Flat Median-7_sub.csv     Rolling Median-28_sub.csv


In [27]:
%%time

#alphas = [1.028, 1.023, 1.018]

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333
0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2

In [28]:
!ls
sub.head(10)

Flat Mean-14_sub.csv
Flat Mean-28_sub.csv
Flat Mean-56_sub.csv
Flat Mean-7_sub.csv
Flat Median-14_sub.csv
Flat Median-28_sub.csv
Flat Median-56_sub.csv
Flat Median-7_sub.csv
M5-Competitors-Guide-Final-10-March-2020.docx
PyProg
RProg
Rolling Mean-14_sub.csv
Rolling Mean-28_sub.csv
Rolling Mean-56_sub.csv
Rolling Mean-7_sub.csv
Rolling Median-14_sub.csv
Rolling Median-28_sub.csv
Rolling Median-56_sub.csv
Rolling Median-7_sub.csv
adata
model.lgb
rawdata
results
submission.csv
submission_0.csv
submission_1.csv
submission_2.csv


F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.868287,0.825110,0.818142,0.810077,1.086353,1.203457,1.180104,0.984156,0.956080,0.952244,0.942797,1.088206,1.313543,1.078247,0.973597,0.909993,0.922834,0.927974,1.044710,1.306228,1.210225,0.943355,0.857020,0.828554,0.851254,1.017645,1.249226,1.230932
1,FOODS_1_001_CA_2_validation,1.027742,1.112136,0.940855,1.293672,1.349351,1.592558,1.413918,0.960485,0.960175,0.917148,0.931625,1.157707,1.586134,1.171868,1.026034,0.988990,1.007631,1.007683,1.246033,1.710998,1.580888,1.044748,0.977176,0.984890,0.962981,1.209919,1.668926,1.451652
2,FOODS_1_001_CA_3_validation,1.111616,1.079880,0.961308,0.942359,1.004658,1.229256,1.347143,1.057719,1.096034,0.970536,1.075435,1.057711,1.363508,1.163533,1.108697,1.084734,0.978990,0.979531,1.085082,1.529958,1.680741,1.104123,1.041452,0.932138,0.922842,1.007811,1.335571,1.297674
3,FOODS_1_001_CA_4_validation,0.403843,0.370191,0.373937,0.384265,0.452046,0.443372,0.526601,0.414692,0.421282,0.412370,0.444215,0.398096,0.431243,0.393774,0.360263,0.373270,0.399011,0.431615,0.451879,0.479253,0.485342,0.376084,0.369549,0.369321,0.376454,0.433138,0.463839,0.478808
4,FOODS_1_001_TX_1_validation,0.167513,0.168283,0.168748,0.173358,0.167412,0.153192,0.210974,0.478386,0.448784,0.415035,0.431267,0.434083,0.435459,0.373025,0.370820,0.425452,0.340063,0.344138,0.406584,0.367235,0.339497,0.291313,0.288410,0.283008,0.282046,0.291854,0.309505,0.309316
5,FOODS_1_001_TX_2_validation,0.437731,0.414052,0.416702,0.385171,0.413247,0.465104,0.542224,0.451617,0.465523,0.438034,0.518533,0.518942,0.585048,0.497177,0.448495,0.433014,0.444060,0.442244,0.496941,0.552735,0.565499,0.420713,0.407533,0.420537,0.405525,0.471607,0.526471,0.522332
6,FOODS_1_001_TX_3_validation,0.370554,0.342149,0.347750,0.409273,0.447342,0.444691,0.545774,0.455114,0.516458,0.474445,0.470116,0.516707,0.542042,0.452959,0.436631,0.425536,0.444721,0.483711,0.500173,0.526137,0.525356,0.426916,0.407794,0.421744,0.425855,0.472134,0.510931,0.495731
7,FOODS_1_001_WI_1_validation,0.324176,0.368307,0.351555,0.370692,0.459628,0.710073,0.600554,0.500851,0.465440,0.446689,0.511744,0.640344,0.770252,0.581715,0.518830,0.521681,0.548778,0.554708,0.674729,0.849536,0.756606,0.543491,0.523605,0.533572,0.543006,0.688588,0.979456,0.779480
8,FOODS_1_001_WI_2_validation,0.308484,0.334902,0.355876,0.355911,0.438282,0.406911,0.409730,0.442501,0.514579,0.421605,0.420573,0.498374,0.504513,0.391525,0.438563,0.417294,0.477861,0.467871,0.467405,0.514163,0.499418,0.374261,0.384123,0.370485,0.379510,0.401131,0.436131,0.408369
9,FOODS_1_001_WI_3_validation,0.237743,0.235036,0.219215,0.226972,0.285946,0.313118,0.366845,0.381684,0.403397,0.386994,0.415707,0.454174,0.427357,0.366931,0.348355,0.344218,0.365442,0.354996,0.407068,0.444255,0.421870,0.322222,0.316169,0.325933,0.323197,0.389611,0.397007,0.349355


In [24]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [25]:
sub.shape

(60980, 29)

In [23]:
import gc
gc.collect()

404